In [ ]:
import random

# Read all tokenized sentences
with open("sentence_tokens.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f if line.strip()]

# Shuffle
random.seed(42)
random.shuffle(sentences)

# Split 80 / 10 / 10
n = len(sentences)
train_end = int(0.8 * n)
val_end = int(0.9 * n)

train_sentences = sentences[:train_end]
val_sentences = sentences[train_end:val_end]
test_sentences = sentences[val_end:]

# Save splits
with open("train.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(train_sentences))
with open("val.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(val_sentences))
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(test_sentences))

print(f"✅ Created train/val/test files successfully!")
print(f"   Total sentences: {n}")
print(f"   Train: {len(train_sentences)} | Val: {len(val_sentences)} | Test: {len(test_sentences)}")


✅ Created train/val/test files successfully!
   Total sentences: 20077
   Train: 16061 | Val: 2008 | Test: 2008


In [ ]:
import random
from collections import Counter
from math import log
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# =====================================================
# STEP 1: Read and Clean Sentences
# =====================================================
with open("sentence_tokens.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f if line.strip()]

# 🧹 Remove duplicates
sentences = list(dict.fromkeys(sentences))

# 🧹 Remove very short or punctuation-only sentences
sentences = [s for s in sentences if len(s.split()) > 2]

# =====================================================
# STEP 2: Shuffle and Split (80/10/10)
# =====================================================
random.seed(42)
random.shuffle(sentences)

n = len(sentences)
train_end = int(0.8 * n)
val_end = int(0.9 * n)

train_sentences = sentences[:train_end]
val_sentences = sentences[train_end:val_end]
test_sentences = sentences[val_end:]

# Save the splits
for name, data in zip(["train", "val", "test"], [train_sentences, val_sentences, test_sentences]):
    with open(f"{name}.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(data))

print(f"✅ Data prepared successfully!")
print(f"Total: {n} | Train: {len(train_sentences)} | Val: {len(val_sentences)} | Test: {len(test_sentences)}")

# =====================================================
# STEP 3: Build Unigram and Bigram Models from Train
# =====================================================
def get_ngrams(sentences, n=1):
    ngrams = []
    for s in sentences:
        tokens = s.split()
        ngrams.extend([" ".join(tokens[i:i+n]) for i in range(len(tokens)-n+1)])
    return Counter(ngrams)

train_unigrams = get_ngrams(train_sentences, 1)
train_bigrams = get_ngrams(train_sentences, 2)
total_unigrams = sum(train_unigrams.values())
total_bigrams = sum(train_bigrams.values())

print(f"Train unigrams: {len(train_unigrams)} | bigrams: {len(train_bigrams)}")

# =====================================================
# STEP 4: Compute PMI for Val/Test Bigrams
# =====================================================
def compute_pmi(sentences, min_freq=3):
    pmi_scores = {}
    for s in sentences:
        tokens = s.split()
        for i in range(len(tokens) - 1):
            w1, w2 = tokens[i], tokens[i + 1]
            bigram = f"{w1} {w2}"
            # Skip unseen or rare bigrams
            if train_bigrams[bigram] < min_freq:
                continue
            p_w1 = train_unigrams[w1] / total_unigrams
            p_w2 = train_unigrams[w2] / total_unigrams
            p_bigram = train_bigrams[bigram] / total_bigrams
            pmi_scores[bigram] = log(p_bigram / (p_w1 * p_w2))
    return sorted(pmi_scores.items(), key=lambda x: -x[1])[:10]

top_pmi_val = compute_pmi(val_sentences)
top_pmi_test = compute_pmi(test_sentences)

print("\n🔹 Top 10 PMI bigrams (Validation):")
for bigram, score in top_pmi_val:
    print(f"{bigram:<30} → PMI = {score:.3f}")

print("\n🔹 Top 10 PMI bigrams (Test):")
for bigram, score in top_pmi_test:
    print(f"{bigram:<30} → PMI = {score:.3f}")

# =====================================================
# STEP 5: TF-IDF Vectorization
# =====================================================
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(train_sentences)
tfidf_val = vectorizer.transform(val_sentences)
tfidf_test = vectorizer.transform(test_sentences)

print("\n✅ TF-IDF vectorization done.")
print(f"Train: {tfidf_train.shape}, Val: {tfidf_val.shape}, Test: {tfidf_test.shape}")

# =====================================================
# STEP 6: Nearest Neighbor Search (Within Set)
# =====================================================
def nearest_neighbors(tfidf_matrix, sentences, top_n=10):
    sims = cosine_similarity(tfidf_matrix)
    np.fill_diagonal(sims, 0)  # ignore self
    pairs = []
    for i in range(len(sentences)):
        j = np.argmax(sims[i])
        pairs.append((sentences[i], sentences[j], sims[i][j]))
    pairs.sort(key=lambda x: -x[2])
    return pairs[:top_n]

print("\n🔹 Top 10 Similar Sentence Pairs (Validation Set):")
for a, b, sim in nearest_neighbors(tfidf_val, val_sentences):
    print(f"{sim:.3f} | {a[:40]} ↔ {b[:40]}")

# =====================================================
# STEP 7: Nearest Neighbor (Val/Test → Train)
# =====================================================
def nearest_in_training(tfidf_train, tfidf_other, other_sentences, train_sentences, top_n=10, batch_size=1000):
    neighbors = []
    for start in range(0, tfidf_other.shape[0], batch_size):
        end = start + batch_size
        sims = cosine_similarity(tfidf_other[start:end], tfidf_train)
        nearest_idx = np.argmax(sims, axis=1)
        max_sims = np.max(sims, axis=1)
        for i, j, s in zip(range(start, end), nearest_idx, max_sims):
            neighbors.append((other_sentences[i], train_sentences[j], s))
    neighbors.sort(key=lambda x: -x[2])
    return neighbors[:top_n]

print("\n🔹 Top 10 Nearest Validation → Train Sentences:")
for a, b, sim in nearest_in_training(tfidf_train, tfidf_val, val_sentences, train_sentences):
    print(f"{sim:.3f} | {a[:40]} ↔ {b[:40]}")

print("\n✅ All tasks completed successfully!")


✅ Data prepared successfully!
Total: 140439 | Train: 112351 | Val: 14044 | Test: 14044
Train unigrams: 81181 | bigrams: 743177

🔹 Top 10 PMI bigrams (Validation):
यूक्रेनियन रिव्निया            → PMI = 13.652
एल्वीरो पीटरसन                 → PMI = 13.141
मेहुल चोकसी                    → PMI = 13.141
मैक्सिकन पेसो                  → PMI = 13.141
रोरी बर्न्स                    → PMI = 13.141
ब्रॉन स्ट्रोमैन                → PMI = 13.141
हरकी पैड़ी                     → PMI = 12.959
सुप्रिया सुले                  → PMI = 12.854
डीवाई चंद्रचूड़                → PMI = 12.854
पल्ला झाड़                      → PMI = 12.805

🔹 Top 10 PMI bigrams (Test):
ग्रेटा थनबर्ग                  → PMI = 13.364
मैक्सिकन पेसो                  → PMI = 13.141
नमामि गंगे                     → PMI = 13.141
जेफ बेजोस                      → PMI = 13.141
ब्रॉन स्ट्रोमैन                → PMI = 13.141
െ ്                            → PMI = 12.959
ऐल्फ्रेड बर्नार्ड              → PMI = 12.854
सुप्रिया सुले            